In [1]:
import sys
import importlib
sys.path.insert(0, ".")

import sql_functions as sf
from sql_functions import *
importlib.reload(sf)

import numpy as np
import pandas as pd

In [2]:
import yaml
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from psycopg2.extras import execute_values

with open("config.yml") as f:
    config = yaml.safe_load(f)

In [3]:
conn = psycopg2.connect(
        database=config['db']['name'], 
        user=config['db']['user'], 
        host=config['db']['host'], 
        password=config['db']['password']
    )

cur = conn.cursor()

In [4]:
import pandas as pd
import requests
from typing import List


class GetSkills:

    endpoint = "https://prodazure-ml-api.skyhive.io/data-preparation" \
                    "/entity/skill"
    skills = requests.get(endpoint).json()


    def filter_skills(self, lang="en") -> List[str]:
        """Load approved skills from a list of dictionaries

        Args:
            skills: a list of individual skill dictionaries

        Returns:
            selected_skills: a list of skill strings
        """
        selected_skills: List[str] = []
        selected_defs = []
        for skill_dict in self.skills:
            approved = skill_dict.get("isApproved", False)
            if approved:
                val = skill_dict.get("title", None)
                if val is not None:
                    skill: str = val.get(lang, None)
                    defin = skill_dict.get("definition", None).get("en")
                    if skill is not None and defin is not None:
                        selected_skills.append(skill)
                        selected_defs.append(defin)
        return selected_skills, selected_defs


    def get_skills_and_synonyms(self):
        selected_skills: List[str] = []
        selected_defs = []
        synonym_coll = []
        ids = []
        for skill_dict in self.skills:
            approved = skill_dict.get("isApproved", False)
            if approved:
                val = skill_dict.get("title", None)
                if val is not None:
                    skill: str = val.get('en', None)
                    defin = skill_dict.get("definition", None).get("en")
                    if skill is not None and defin is not None:
                        selected_skills.append(skill)
                        selected_defs.append(defin)
                        skill_id = skill_dict['id']
                        ids.append(skill_id)
                        synonyms = skill_dict['synonyms']
                        synonym_coll.append(synonyms)
        return pd.DataFrame({
            'id': ids,
            'skill': selected_skills,
            'synonyms': synonym_coll,
            'definition': selected_defs
        })

    def get_multilingual_skills(self):
        skills_en: List[str] = []
        skills_fr: List[str] = []
        skills_fr_ca: List[str] = []
        skills_es: List[str] = []
        skills_ja: List[str] = []
        skills_ko: List[str] = []
        skills_it: List[str] = []
        definitions_en: List[str] = []
        definitions_fr: List[str] = []
        definitions_fr_ca: List[str] = []
        definitions_es: List[str] = []
        definitions_ja: List[str] = []
        definitions_ko: List[str] = []
        definitions_it: List[str] = []
        synonym_coll = []
        extractions_en = []
        extractions_fr = []
        extractions_fr_ca = []
        extractions_es = []
        extractions_ja = []
        extractions_ko = []
        extractions_it = []
        all_extractions = [
            extractions_en,
            extractions_fr,
            extractions_fr_ca,
            extractions_es,
            extractions_ja,
            extractions_ko,
            extractions_it
        ]
        ids = []
        for skill_dict in self.skills:
            approved = skill_dict.get("isApproved", False)
            if approved:
                val = skill_dict.get("title", None)
                if val is not None:
                    skill_en: str = val.get('en', None)
                    skill_fr: str = val.get('fr', None)
                    skill_fr_ca: str = val.get('fr-ca', None)
                    skill_es: str = val.get('es', None)
                    skill_ja: str = val.get('ja', None)
                    skill_ko: str = val.get('ko', None)
                    skill_it: str = val.get('it', None)
                    definition_dict = skill_dict.get("definition", None)
                    definition_en = definition_dict.get("en")
                    definition_fr = definition_dict.get("fr")
                    definition_fr_ca = definition_dict.get("fr-ca")
                    definition_es = definition_dict.get("es")
                    definition_ja = definition_dict.get("ja")
                    definition_ko = definition_dict.get("ko")
                    definition_it = definition_dict.get("it")
                    extraction_forms = skill_dict.get("extractionTitle")
                    #if skill_en is not None and definition_en is not None:
                    if skill_en is not None:
                        skills_en.append(skill_en)
                        skills_fr.append(skill_fr)
                        skills_fr_ca.append(skill_fr_ca)
                        skills_es.append(skill_es)
                        skills_ja.append(skill_ja)
                        skills_ko.append(skill_ko)
                        skills_it.append(skill_it)
                        definitions_en.append(definition_en)
                        definitions_fr.append(definition_fr)
                        definitions_fr_ca.append(definition_fr_ca)
                        definitions_es.append(definition_es)
                        definitions_ja.append(definition_ja)
                        definitions_ko.append(definition_ko)
                        definitions_it.append(definition_it)
                        skill_id = skill_dict['id']
                        ids.append(skill_id)
                        synonyms = skill_dict['synonyms']
                        synonym_coll.append(synonyms)
                        if extraction_forms is None:
                            for arr in all_extractions:
                                arr.append(None)
                            continue
                        else:
                            extracted_en = extraction_forms.get('en')
                            extractions_en.append(extracted_en)
                            extracted_fr = extraction_forms.get('fr')
                            extractions_fr.append(extracted_fr)
                            extracted_fr_ca = extraction_forms.get('fr_ca')
                            extractions_fr_ca.append(extracted_fr_ca)
                            extracted_es = extraction_forms.get('es')
                            extractions_es.append(extracted_es)
                            extracted_ja = extraction_forms.get('ja')
                            extractions_ja.append(extracted_ja)
                            extracted_ko = extraction_forms.get('ko')
                            extractions_ko.append(extracted_ko)
                            extracted_it = extraction_forms.get('it')
                            extractions_it.append(extracted_it)
        return pd.DataFrame({
            'id': ids,
            'skill': skills_en,
            'skill_fr': skills_fr,
            'skill_fr_ca': skills_fr_ca,
            'skill_es': skills_es,
            'skill_it': skills_it,
            'skill_jp': skills_ja,
            'skill_kr': skills_ko,
            'synonyms': synonym_coll,
            'definition': definitions_en,
            'definition_fr': definitions_fr,
            'definition_fr_ca': definitions_fr_ca,
            'definition_es': definitions_es,
            'definition_it': definitions_it,
            'definition_jp': definitions_ja,
            'definition_kr': definitions_ko,
            'extraction_en': extractions_en,
            'extraction_fr': extractions_fr,
            'extraction_fr_ca': extractions_fr_ca,
            'extraction_es': extractions_es,
            'extraction_it': extractions_it,
            'extraction_jp': extractions_ja,
            'extraction_kr': extractions_ko
        })

In [5]:
# Get sql skill that doesn't show on B2C
def get_not_in_mongo_df(title):
    not_in_mongo_df = pd.read_sql(f"select * from skills_ja_jp where skill = %s", conn, params=(title, ))
    return not_in_mongo_df

In [6]:
# Get title_en_us by kill_id
def get_title_en_by_skill_id(skill_id):
    get_title_en_by_skill_id = pd.read_sql(f"select skill from skills_en_us where skill_id = %s", conn, params=(skill_id, ))
    return get_title_en_by_skill_id

In [7]:
#Create a dataframe of serial_ids
def get_sql_serial_id_df(skill):
    #sql_serial_id_df = pd.read_sql(f"select serial_id from skills where skill = %s", conn, params=(skill, ))
    sql_serial_id_df = pd.read_sql(f"select serial_id from skills where skill = '{skill}'", conn)
    return sql_serial_id_df

In [8]:
#Create a df with data from Mongo db
mongo_df = GetSkills().get_multilingual_skills()
mongo_df

,id,skill,skill_fr,skill_fr_ca,skill_es,skill_it,skill_jp,skill_kr,synonyms,definition,...,definition_it,definition_jp,definition_kr,extraction_en,extraction_fr,extraction_fr_ca,extraction_es,extraction_it,extraction_jp,extraction_kr
0,614e3feb386e6247a32cf5b8,.net Architecture,None,Architecture .net,Arquitectura .NET,None,None,.net Architecture,,None,...,None,None,None,None,None,None,None,None,None,None
1,5ec5dd60bd86b3287e0f992d,.NET Assemblies,Assemblages .NET,.NET Assemblies,Ensamblajes .NET,.NET Assemblies,.NETアセンブリ,.NET Assemblies,,Defined by Microsoft for use in recent version...,...,None,最近のバージョンのWindowsで使用するためにMicrosoftによって定義された、共通言...,최신 버전의 Windows에서 사용하기 위해 Microsoft에서 정의한 CLI(공...,.NET Assemblies,.NET Assemblies,None,None,None,None,None
2,5b1b16e8a0ad036875697504,.NET Framework,.NET Framework,Cadre .NET,.NET Framework,.NET Framework,.NET Framework,.NET Framework,".net; .NET, .Net",.NET Framework is a software framework develop...,...,None,.NET Frameworkは、主にMicrosoftWindows上で実行されるMicro...,.NET Framework는 주로 Microsoft Windows에서 실행되는 Mi...,.NET Framework,None,None,None,None,None,None
3,5ec5ddb0bd86b3287e0f9938,.NET Reflector,Réflecteur .NET,.NET Reflector,Reflector .NET,.NET Reflector,.NETリフレクター,.NET Reflector,,".NET Reflector is a class browser, decompiler ...",...,None,.NET Reflectorは、元々Lutz Roederによって作成された、.NET Fr...,.NET Reflector는 원래 Lutz Roeder가 작성한 .NET Frame...,.NET Reflector,None,None,None,None,None,None
4,5ec5ddb0ec223f743f884251,.NET Remoting,.NET Remoting,.NET Remoting,.NET remoto,.NET Remoting,.NET Remoting,.NET Remoting,,.NET Remoting is a Microsoft application progr...,...,None,.NET Remotingは、.NETFrameworkの1.0バージョンで2002年にリリ...,.NET Remoting은 .NET Framework의 1.0 버전과 함께 2002...,.NET Remoting,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12145,5ec5e6c14d0455767ecf299b,Zone File,Fichier de zone,Fichier zone,Archivo de zona,File di zona,ゾーンファイル,영역 파일,,A Domain Name System zone file is a text file ...,...,None,ドメインネームシステムゾーンファイルは、DNSゾーンを説明するテキストファイルです。 DNS...,도메인 이름 시스템 영역 파일은 DNS 영역을 설명하는 텍스트 파일입니다. DNS ...,Zone File,None,None,None,None,None,None
12146,5ebc13084d679d1f836681af,Zoning,Zonage,Zonage,Zonificación,Zonizzazione,ゾーニング,영역 지정,Rezoning; Land Zoning,Zoning is a method of urban planning in which ...,...,None,ゾーニングは、自治体または政府の他の層が土地をゾーンと呼ばれる領域に分割し、その中で特定の土...,구역 설정은 지방 자치 단체 또는 기타 정부 계층에서 토지를 특정 토지 사용이 허용...,None,Zonage,None,None,None,None,None
12147,5a9cd81dfc0da84a0c1c28ac,Zoology,Zoologie,Zoologie,Zoología,Zoologia,動物学,동물학,,Zoology is the branch of biology that studies ...,...,None,動物学は、生きている動物と絶滅した動物の両方の構造、発生学、進化、分類、習慣、分布、およびそ...,"동물학은 구조, 발생학, 진화, 분류, 습성 및 분포를 포함하는 동물의 왕국을 연구...",Zoology,Zoologie,None,None,None,None,None
12148,5ec5e6c1bd86b3287e0ff00b,Zotero,Zotero,Zotero,Zotero,Zotero,Zotero,Zotero,,Zotero is a free and open-source reference man...,...,None,Zoteroは、書誌データと関連する研究資料を管理するための無料のオープンソース参照管理ソフ...,Zotero는 서지 데이터 및 관련 연구 자료를 관리하기 위한 무료 오픈 소스 참조...,Zotero,None,None,None,None,None,None


In [9]:
# Create a df with data from SQL database where data stamp is greater than Oct 2022.
sql_df = pd.read_sql(f"select * from skills_ja_jp where timestamp >= '2022-10-01 00:00:00'", conn)
sql_df

C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,timestamp,skill,base_form,skill_definition,skill_id
0,11837,2023-01-19 01:16:13.246072,貨物輸送 (Freight Transport),None,None,4365
1,11841,2023-01-19 01:16:20.323529,オーケストラの演出,None,None,7927
2,11842,2023-01-19 01:16:22.502051,航空機保守,None,None,332
3,11845,2023-01-19 01:16:28.115458,宅配便,None,None,8053
4,11848,2023-01-19 01:16:35.744758,変更管理,None,None,1763
...,...,...,...,...,...,...
557,11830,2023-01-10 23:31:55.862545,データのサニタイズ,None,データのサニタイズは、データライフサイクルの重要なステップです。データが寿命の終わりに達した...,12318
558,11831,2023-01-10 23:31:55.862545,文字エンコーディング,None,文字エンコーディングは、テキストデータをバイナリ番号に変換する手法です。一意の数値値を特定の...,12319
559,11832,2023-01-10 23:31:55.862545,デシリアリゼーション,None,デシリアリゼーションとは、オブジェクトをインスタンス化するために、一連のバイトまたは文字列か...,12320
560,11833,2023-01-10 23:31:55.862545,モジュラープログラミング,None,モジュラープログラミングは、プログラムの機能を独立した交換可能なモジュールに分離することを強...,12321


In [10]:
not_in_mongodb = []

for skill in sql_df['skill'].tolist():
    if skill not in mongo_df['skill_jp'].tolist():
        not_in_mongodb.append(skill)

In [11]:
len(not_in_mongodb)

22

In [12]:
not_in_mongodb

['制御フロー',
 '運用システムのセキュリティ',
 '配列（データ構造）',
 'リソースリーク',
 'プログラミングパラダイム',
 'jlink',
 'コンフィギュレーション',
 'Java データ構造',
 '継承（オブジェクト指向プログラミング）',
 '例外処理',
 '文字列メソッド',
 'Javaクラスライブラリ',
 'メソッド（コンピュータープログラミング）',
 'Java num型',
 'メモリアーキテクチャ',
 'スレッド（コンピューティング）',
 'コンピューターメモリ',
 'データのサニタイズ',
 '文字エンコーディング',
 'デシリアリゼーション',
 'モジュラープログラミング',
 '取材']

In [13]:
results_df = pd.DataFrame()

In [14]:
for row in not_in_mongodb:
    result = get_not_in_mongo_df(row)
    results_df = pd.concat([results_df, result], axis=0)

C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordat

In [15]:
results_df

,id,timestamp,skill,base_form,skill_definition,skill_id
0,11826,2023-01-10 23:31:55.862545,制御フロー,None,コンピューターサイエンスでは、制御フロー（または制御の流れ）とは、命令プログラムの個々のステ...,12314
0,11798,2023-01-10 19:17:31.580162,運用システムのセキュリティ,None,運用システムのセキュリティ（OSセキュリティ）は、OSの整合性、機密性、および可用性を確保す...,12286
0,11825,2023-01-10 23:31:55.862545,配列（データ構造）,None,コンピューターサイエンスでは、配列は要素のコレクション（値または変数）で構成されるデータ構造...,12313
0,11806,2023-01-10 19:17:31.580162,リソースリーク,None,コンピューターサイエンスでは、リソースリークは、プログラムが取得したリソースをリリースしない...,12294
0,11807,2023-01-10 19:17:31.580162,プログラミングパラダイム,None,プログラミングパラダイムは、機能に基づいてプログラミング言語を分類する方法です。言語は複数の...,12295
0,11815,2023-01-10 19:17:31.580162,jlink,None,Jlinkは、カスタムJavaランタイム環境（JRE）を生成するために使用されるJavaコマ...,12303
0,11817,2023-01-10 23:31:55.862545,コンフィギュレーション,None,アプリケーション構成は、構成ファイルなどのいくつかのメカニズムを介して、特定の方法でその動作...,12305
0,11818,2023-01-10 23:31:55.862545,Java データ構造,None,コンピューターサイエンスおよびコンピュータープログラミングでは、データ型（または単にタイプ）...,12306
0,11819,2023-01-10 23:31:55.862545,継承（オブジェクト指向プログラミング）,None,オブジェクト指向プログラミングでは、継承は、別のオブジェクト（プロトタイプベースの継承）また...,12307
0,11820,2023-01-10 23:31:55.862545,例外処理,None,コンピューティングとコンピュータープログラミングでは、例外処理は、プログラムの実行中に例外（...,12308


In [16]:
# Get title_en conresponding to  results_df['skill']

title_en_df = pd.DataFrame()

for row in results_df['skill_id'].tolist():
    result = get_title_en_by_skill_id(row)
    if not result.empty:
        title_en_df  = pd.concat([title_en_df, result], axis=0)
    if result.empty:
        empty_cell_df = pd.DataFrame({'skill':['none']})
        title_en_df  = pd.concat([title_en_df, empty_cell_df], axis=0)

C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordat

In [17]:
title_en_df

,skill
0,Control Flow
0,Operating System Security
0,Array (Data Structure)
0,Resource Leak
0,Programming Paradigm
0,jlink
0,Application Configuration
0,Data Type
0,Inheritance (Object-oriented Programming)
0,Exception Handling


In [26]:
combined_df = pd.DataFrame({'id':results_df['id'], 'skill_id':results_df['skill_id'], 'title_en':title_en_df['skill'],'title_local':results_df['skill'],'skill_def':results_df['skill_definition'] })
combined_df

,id,skill_id,title_en,title_local,skill_def
0,11826,12314,Control Flow,制御フロー,コンピューターサイエンスでは、制御フロー（または制御の流れ）とは、命令プログラムの個々のステ...
0,11798,12286,Operating System Security,運用システムのセキュリティ,運用システムのセキュリティ（OSセキュリティ）は、OSの整合性、機密性、および可用性を確保す...
0,11825,12313,Array (Data Structure),配列（データ構造）,コンピューターサイエンスでは、配列は要素のコレクション（値または変数）で構成されるデータ構造...
0,11806,12294,Resource Leak,リソースリーク,コンピューターサイエンスでは、リソースリークは、プログラムが取得したリソースをリリースしない...
0,11807,12295,Programming Paradigm,プログラミングパラダイム,プログラミングパラダイムは、機能に基づいてプログラミング言語を分類する方法です。言語は複数の...
0,11815,12303,jlink,jlink,Jlinkは、カスタムJavaランタイム環境（JRE）を生成するために使用されるJavaコマ...
0,11817,12305,Application Configuration,コンフィギュレーション,アプリケーション構成は、構成ファイルなどのいくつかのメカニズムを介して、特定の方法でその動作...
0,11818,12306,Data Type,Java データ構造,コンピューターサイエンスおよびコンピュータープログラミングでは、データ型（または単にタイプ）...
0,11819,12307,Inheritance (Object-oriented Programming),継承（オブジェクト指向プログラミング）,オブジェクト指向プログラミングでは、継承は、別のオブジェクト（プロトタイプベースの継承）また...
0,11820,12308,Exception Handling,例外処理,コンピューティングとコンピュータープログラミングでは、例外処理は、プログラムの実行中に例外（...


In [27]:
# Get SQL serial_id

sql_serial_id_df = pd.DataFrame()

for row in combined_df['title_en'].tolist():
    result  = get_sql_serial_id_df(row)
    if not result.empty:
        sql_serial_id_df = pd.concat([sql_serial_id_df, result], axis=0)
    if result.empty:
        empty_cell_df = pd.DataFrame({'serial_id':['none']})
        sql_serial_id_df = pd.concat([sql_serial_id_df, empty_cell_df], axis=0)

C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\KeikoGolden\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordat

In [28]:
sql_serial_id_df

,serial_id
0,63bdf56383dfb3438457ac4e
0,63bdb9c383dfb32b54855e4b
0,63bdf56383dfb3438457ac4d
0,63bdb9c383dfb32b54855e53
0,63bdb9c383dfb32b54855e54
0,63bdb9c383dfb32b54855e5c
0,63bdf56383dfb3438457ac45
0,63bdf56383dfb3438457ac46
0,63bdf56383dfb3438457ac47
0,63bdf56383dfb3438457ac48


In [29]:
#Get Mongo serial_id
mongo_serial_id = []

for i, row in enumerate(combined_df['title_en'].tolist()):
    if row in mongo_df['skill'].tolist():
        #index = list(mongo_df.reset_index().query('skill == "{row}"').index) #インデックスが番号でなくて名前になってる場合はreset_index()を.queryの前に使う。
        index = list(mongo_df.query(f'skill == "{row}"').index)
        result = list(mongo_df.loc[index, 'id']) #list()でもtuple()でもどちらを使っても可。両者ともSerial番号のみを引き出す。
        mongo_serial_id.append(result)
    else:
        result = ''
        mongo_serial_id.append(result)

In [30]:
mongo_serial_id

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ['638fd53bb23c6c44142ee5cc']]

In [33]:
combined_df = pd.DataFrame({'id':results_df['id'], 'skill_id':results_df['skill_id'], 'sql_serial_id':sql_serial_id_df['serial_id'], 'mongo_serial_id':mongo_serial_id, 'title_en':title_en_df['skill'],'title_local':results_df['skill'],'skill_def':results_df['skill_definition'] })
combined_df

,id,skill_id,sql_serial_id,mongo_serial_id,title_en,title_local,skill_def
0,11826,12314,63bdf56383dfb3438457ac4e,,Control Flow,制御フロー,コンピューターサイエンスでは、制御フロー（または制御の流れ）とは、命令プログラムの個々のステ...
0,11798,12286,63bdb9c383dfb32b54855e4b,,Operating System Security,運用システムのセキュリティ,運用システムのセキュリティ（OSセキュリティ）は、OSの整合性、機密性、および可用性を確保す...
0,11825,12313,63bdf56383dfb3438457ac4d,,Array (Data Structure),配列（データ構造）,コンピューターサイエンスでは、配列は要素のコレクション（値または変数）で構成されるデータ構造...
0,11806,12294,63bdb9c383dfb32b54855e53,,Resource Leak,リソースリーク,コンピューターサイエンスでは、リソースリークは、プログラムが取得したリソースをリリースしない...
0,11807,12295,63bdb9c383dfb32b54855e54,,Programming Paradigm,プログラミングパラダイム,プログラミングパラダイムは、機能に基づいてプログラミング言語を分類する方法です。言語は複数の...
0,11815,12303,63bdb9c383dfb32b54855e5c,,jlink,jlink,Jlinkは、カスタムJavaランタイム環境（JRE）を生成するために使用されるJavaコマ...
0,11817,12305,63bdf56383dfb3438457ac45,,Application Configuration,コンフィギュレーション,アプリケーション構成は、構成ファイルなどのいくつかのメカニズムを介して、特定の方法でその動作...
0,11818,12306,63bdf56383dfb3438457ac46,,Data Type,Java データ構造,コンピューターサイエンスおよびコンピュータープログラミングでは、データ型（または単にタイプ）...
0,11819,12307,63bdf56383dfb3438457ac47,,Inheritance (Object-oriented Programming),継承（オブジェクト指向プログラミング）,オブジェクト指向プログラミングでは、継承は、別のオブジェクト（プロトタイプベースの継承）また...
0,11820,12308,63bdf56383dfb3438457ac48,,Exception Handling,例外処理,コンピューティングとコンピュータープログラミングでは、例外処理は、プログラムの実行中に例外（...


In [34]:
combined_df.to_excel('ja_new_skill_gap.xlsx')